<a href="https://colab.research.google.com/github/AlexAlexeyuk/Deal_with_epicrisis/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22Fbrngn_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from re import sub, search, findall
import os
import pandas as pd
import numpy as np
from glob import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
lst = list(glob(os.path.join('/content/drive/MyDrive/unzipped_and_converted', '*')))

In [31]:
lst = lst[:20]

In [32]:
def remover(file_):
  with open(file_, encoding='utf-8', errors='ignore') as f:
    file_ = ''.join(f.read().split()).lower()
    stop_element = [',', 'от', 'менее', '<', '>']
    for elem in stop_element:
      if elem == ',':
        file_ = sub(elem, '.', file_)
      else:
        file_ = sub(elem, '', file_)
    return(file_)

In [33]:
def change_date_format(dt):
    """ на первом этапе переводим "нормальный формат" в формат через дефисы
     на втором этапе ищем вариант дд.мм.гг в формат через дефисы"""
    data_1 =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', dt)
    return sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})', 
                  '\\1-\\2-20\\3', data_1)

In [35]:
list(map(change_date_format,list(map(remover, lst))))

In [36]:
print(ll[1])

﻿учреждениездравоохранения"гродненскаяуниверситетскаяклиника"эндокринологическоеделениеучреждениездравоохранения"гродненскаяуниверситетскаяклиника"эпикризизмедицинскойкартыстационарногопациента№10216ф.и.о:крыловвадимвикторович.03-05-1997местожительства:доваторапер..6-б-31.г.гродно.гродненскаяобл..беларусь.проживаеткироваул..1.г.гродно.гродненскаяобл..беларусьместорабы:нерабает.28-04-2021доставленизтюрьмы№1г.гроднопоступил:28-04-2021выписан:03-05-2021эндокринологическоеделениедиагнозприпоступлении:e10.1сахарныйдиабет.впервыевыявленныйвст.декомпенсации.кетоацидоз./(основной)/хроническийпанкреатит.диагноззаключительный:e10.1сахарныйдиабет1типавпервыевыявленный/(основной)/множественныйкариес./(сопутствующий)/состояниеижалобыприпоступлении:жалобыпациента:наобщуюслабость.сухьворту.головнуюболь.анамнеззаболевания:втечении5днеймечаетухудшениесостояния.когдапоявилисьвышеперечисленныежалобы.корыесопровождалисьбольювживе.резкойпереймассытела.анамнезжизни:туберкулез.вир.гепатит.вен.заболеваниярица

In [25]:
print(list(ll))

['\ufeffучреждениездравоохранения"гродненскаяуниверситетскаяклиника"микрохирургияглазаучреждениездравоохранения"гродненскаяуниверситетскаяклиника"эпикризизмедицинскойкартыстационарногопациента№10494ф.и.о:кучинскийюрийиосифович.02.09.1970местожительства:лиможаул..45-2-37.г.гродно.гродненскаяобл..беларусьместорабы:ггуппспецавтохозяйствопоступил:03.05.2021выписан:12.05.2021микрохирургияглазадиагнозприпоступлении:h26.2осложненнаякатарактаправогоглаза./(основной)/диагноззаключительный:h26.2осложненнаякатарактаправогоглаза./(основной)/состояниеижалобыприпоступлении:жалобынананизкуюострузрениянаправомглазу.анамнеззаболевания:зрениенаправомглазуснизилосьвтечениипоследнегогода.лечилсяамбулааторнопом/ж.капалантикатарактальныекапли.поступилдляплановогооперативноголечениякатарактыправогоглазаперенесенныезаболевания:гэраллергологическийанамнез:неягощентрансфузиологическийанамнез:неягощенэпидемиологическийанамнез:вконтактесинфекционнымбольным.втомчислеcovid-19небыл.объективныйстатусобщеесостояниеудо

In [ ]:
# было бы лучше здесь инициализировать функцию change_date_format, и затем ей пользоваться,
# просто в таком варианте придётся несколько раз использовать, учитывая, что мы делаем таблицу для каждого значения. 
# надо стараться давать говорящие что-то имена в циклах фор 
# желательно сделать функцию, которая делает всю грязную работу сразу: удаляет запятые, чистит пробелы, убирает стоп слова
# типа: от, менее, за, знаки > <  и проч. 

In [ ]:
lst = list(glob(os.path.join('/content/drive/MyDrive/unzipped_and_converted', '*')))
dates = []
values = []
id_list = []
for doc in lst:
   with open(doc, encoding='utf-8', errors='ignore') as f:
     file_ = (sub(r',', '.', ''.join(f.read().split()).lower())
     file_ = sub(r'от', '', file_)
     file_ =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', file_)
     file_ = sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})',
                    '\\1-\\2-20\\3', file_)
     file_ = sub(r'(\d{2}-\d{2}-\d{4})(гемостазиограмма)', r'\2\1', file_) #ставим дату после гемостазиограммы, чтобы проще было искать по одному шаблону.
     for date in findall(r'(?<=гемостазиограмма)(\d{2}-\d{2}-\d{4}|)', file_): #поиск дат, которые идут после гемост-ы.
       dates.append(date)
       id = search(r'\d+', file_) #ищем айди. находит самую первую дату в эпикризе. !!! есть эпикризы в которых первые цифры это номер отделения, типа онко отд. №2
       if id:
         id_list.append(id[0])
     file_1 = sub(r'(гемостазиограмма)(\d{2}-\d{2}-\d{4})', r'\2\1', file_) #здесь наоборот ставим дату ПЕРЕД гемост-ой, чтобы она нам не мешала. тк вариант типа "гемостазиограмма27-09-2021:ачтв.." не даст результата.
     for value in findall(r'(гемостазиограмма.*?)(?=\d{2}-\d{2}-\d{4})', file_1): #задаем поиск строки от гемост-ы до даты другого исследования, при этом НЕ захватываются данные других иссл-ий.
       value = sub(r'(.*)(фибриноген)(\d.\d*)', r'\3\2\1', value) #в найденной строке меняем местами необходимые данные. новая строка имеет вид: 4,66фибриногенгемостазиограмма:ачтв24мно1,20...
       value = sub(r'(фибриноген|гемостазиограмма).*', '', value) #удаляем все, что после фибриногена и/или гемост-ы включительно, оставляя лишь число. если его нет - пустая строка.
       values.append(n) # а здесь почему-то не могу применить float/np.float. хотя во всем числах запятые заменены на точки. пишет - ValueError: could not convert string to float: 

In [ ]:
data_dct = {'ID':id_list, 'dates':dates,'values':values}
df = pd.DataFrame(data_dct)
df.replace(r'', np.nan, regex=True)

,ID,dates,values
0,2839,29-06-2020,7.64
1,2839,06-07-2020,5.31
2,2839,08-07-2020,5.56
3,2839,14-07-2020,3.69
4,2839,17-07-2020,NaN
...,...,...,...
177,2940,23-07-2020,5.1
178,3535,07-09-2020,2.34
179,3535,10-09-2020,6.16
180,3535,14-09-2020,6.6


In [ ]:
df.to_csv('data.csv')